In [65]:
import agate
text = agate.Text()
number = agate.Number()
tester = agate.TypeTester(force={
        'WFO': text,
        'DAMAGE_PROPERTY': text,
        'DAMAGE_CROPS': text,
        'EPISODE_ID': text,
        'INJURIES_INDIRECT': number,
        'DEATHS_DIRECT': number,
        'DEATHS_INDIRECT': number,
        'STATE_FIPS': text,
        'EVENT_ID': text,
    },limit=300)
mean_temps = agate.Table.from_csv('mon_temps_lnk.csv')

In [66]:
print(mean_temps)
mon_averages = {'JAN':24.6, 'FEB':29,'MAR':40.1,'APR':51.6,'MAY':62.3,'JUN':72.6,'JUL':77.6,'AUG':75.3,'SEP':66,'OCT':53.2,'NOV':38.9,'DEC':26.8,'Annual':51.6}
mon_temps = mean_temps.where(lambda row: row['Year'] != 'Averages')
print(len(mon_temps.rows))
mon_temps.print_table(50)

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  Year         | Text          |
|  JAN          | Number        |
|  FEB          | Number        |
|  MAR          | Number        |
|  APR          | Number        |
|  MAY          | Number        |
|  JUN          | Number        |
|  JUL          | Number        |
|  AUG          | Number        |
|  SEP          | Number        |
|  OCT          | Number        |
|  NOV          | Number        |
|  DEC          | Number        |
|  Annual       | Number        |
|---------------+---------------|

129
|-------+------+------+------+------+------+------+------+------+------+------+------+------+---------|
|  Year |  JAN |  FEB |  MAR |  APR |  MAY |  JUN |  JUL |  AUG |  SEP |  OCT |  NOV |  DEC | Annual  |
|-------+------+------+------+------+------+------+------+------+------+------+------+------+---------|
|  2015 | 28.1 | 21.6 | 43.3 | 54.0 | 61.5 | 73.4 | 77.1 | 73.8 | 71.4 

In [77]:

from decimal import Decimal
class differenceAverage(agate.Computation):
    """
    Calculates the difference between normal and each monthly
    """
    def __init__(self, month, monthDict):
        self._month = month
        self._monthDict = monthDict

    def get_computed_data_type(self, table):
        """
        The return value is a number.
        """
        return agate.Number()

    def validate(self, table):
        """
        Verify the column is a number.
        """
        column = table.columns[self._month]

        if not isinstance(column.data_type, agate.Number):
            raise agate.DataTypeError('Can only be applied to number data.')

    def run(self, table):
        new_column = []    
        for row in table.rows:
            current = row[self._month]
            normal = self._monthDict[self._month]
            normal_decimal = Decimal(normal)
            diff = current - normal_decimal
            new_column.append(diff.quantize(Decimal('0.01')))
        return new_column

In [78]:
computations = []
for month in mon_averages.keys():
    print(month)
    value = differenceAverage(month, mon_averages)
    computations.append(('diff_{}'.format(month), value))
new_table = mon_temps.compute(computations)
new_table.print_table()
tables_diff = new_table.select(['Year', 'diff_JAN', 'diff_FEB', 'diff_MAR', 'diff_APR', 'diff_MAY', 'diff_JUN', 'diff_JUL', 'diff_AUG', 'diff_SEP', 'diff_OCT', 'diff_NOV', 'diff_DEC', 'diff_Annual'])

NOV
JAN
JUL
JUN
AUG
MAR
FEB
DEC
APR
MAY
SEP
OCT
Annual
|-------+------+------+------+------+------+------+------+------+------+------+------+------+--------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------------|
|  Year |  JAN |  FEB |  MAR |  APR |  MAY |  JUN |  JUL |  AUG |  SEP |  OCT |  NOV |  DEC | Annual | diff_NOV | diff_JAN | diff_JUL | diff_JUN | diff_AUG | diff_MAR | diff_FEB | diff_DEC | diff_APR | diff_MAY | diff_SEP | diff_OCT | diff_Annual  |
|-------+------+------+------+------+------+------+------+------+------+------+------+------+--------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------------|
|  2015 | 28.1 | 21.6 | 43.3 | 54.0 | 61.5 | 73.4 | 77.1 | 73.8 | 71.4 | 57.1 | 44.9 | 33.6 |   53.3 |      6.0 |      3.5 |     -0.5 |      0.8 |     -1.5 |      3.2 |     -7.4 |      6.8 |     

In [79]:
tables_diff.print_table()

|-------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------------|
|  Year | diff_JAN | diff_FEB | diff_MAR | diff_APR | diff_MAY | diff_JUN | diff_JUL | diff_AUG | diff_SEP | diff_OCT | diff_NOV | diff_DEC | diff_Annual  |
|-------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------------|
|  2015 |      3.5 |     -7.4 |      3.2 |      2.4 |     -0.8 |      0.8 |     -0.5 |     -1.5 |      5.4 |      3.9 |      6.0 |      6.8 |         1.7  |
|  2014 |     -2.3 |     -6.5 |     -3.0 |      1.1 |      2.9 |      1.3 |     -2.6 |      0.8 |     -0.4 |      2.0 |     -4.3 |      4.6 |        -0.5  |
|  2013 |      0.6 |      0.9 |     -5.3 |     -5.5 |     -0.8 |     -0.8 |     -1.8 |      0.8 |      4.6 |     -1.5 |     -2.2 |     -4.6 |        -1.4  |
|  2012 |      5.4 |      1.6 |     14.9 |      4.4 |     

In [81]:
differences = tables_diff.to_csv('lnkMonths.csv')

In [92]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set(style="ticks", context='talk', font_scale=1)
%matplotlib inline


In [93]:
df = pd.read_csv('lnkMonths.csv')

In [117]:
sns.set_style("white")
sns.scatterplot(x="Year" , y="diff_Annual", data=df, aspect=1.5);
plt.ylabel('')
plt.xlabel('Mountain Lion Sightings')
plt.xticks(rotation=90)
#plt.autoscale(tight=True)
ax = plt.gca()
#ax.get_xticks().get_major_formatter().scaled[365] = '%y'
sns.despine(bottom=True, left=True)

AttributeError: module 'seaborn' has no attribute 'scatterplot'